<a href="https://colab.research.google.com/github/xin-2001/taica_1132_GenAI/blob/main/hw6/113_2GenAI_HW6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 主題二：製作兩個聊天機器人互相對話
- 主題一
  
  延續上週作業，參考老師的範例，更改成可以持續對話的版本。
Gradio展示。
- 主題二
  
  製作兩個不同模型互相對話的機器人。
  
  Gradio展示。

- 繳交內容
  
  colab連結(請記得將共用權限打開)，請在重點處以 MarkDown 註明。
  
  此份作業的重點截圖
  
  人設/背景設定
  
  使用的模型
  
  Gradio的對話結果
  
  其他
  
  也可以自行增加其他內容

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

## Ollama

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [1]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

**如果後面運行gradio有暫停運行，可能也會一併將ollama sever暫停，因此要在啟用要重新執行這一行。**

In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


## 機器人1
ollama library:https://ollama.com/library

In [3]:
!ollama pull gemma3:4b # gemma3:1b
model1 = "gemma3:4b"

Error: could not connect to ollama app, is it running?


In [4]:
import openai
from openai import OpenAI

In [5]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [6]:
client1 = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

In [7]:
system1 = "你是一個友善的對話機器人，負責回答使用者的提問，回答需根據受試者的提問層次，從簡單回答開始，逐步深化內容，避免一次性提供完整答案。回答方式需以提示或建議為主，讓受試者透過提問獲取關鍵資訊。回答時請使用台灣用語及繁體中文。"

## 機器人2

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [8]:
!pip install aisuite[all]

In [9]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


In [10]:
model2 = "ollama:gemma3:1b"
system2 = "你是一個對任何事物都充滿好奇，喜歡追根究底，不斷追問「為什麼」，直到得到滿意的答案為止。回答時請使用台灣用語及繁體中文。"

In [11]:
import aisuite as ai

In [12]:
client2 = ai.Client()

## 兩個機器人對話

In [13]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


初始化兩個機器人

In [20]:
# 機器人1
prompt1 = "聊聊喜歡的動物" # 設置對話話題
messages1 = [{"role":"system", "content":system1},
            {"role": "user", "content":prompt1}]
# 機器人2
prompt2 = ""
messages2 = [
    {"role": "system", "content": system2},
    {"role": "user", "content": prompt2},
]

In [21]:
for i in range(6): # 設置對話輪數
  # AI 機器人 1 的回覆
  response1 = client1.chat.completions.create(
      model=model1,
      messages=messages1
  )
  reply1 = response1.choices[0].message.content
  messages2.append({"role": "user", "content": reply1}) # 將回覆添加到 AI 機器人 2 的訊息中


  # AI 機器人 2 的回覆
  response2 = client2.chat.completions.create(
      model=model2,
      messages=messages2
  )
  reply2 = response2.choices[0].message.content
  messages1.append({"role": "user", "content": reply2}) # 將回覆添加到 AI 機器人 1 的訊息中

  print(f"回合 {i+1}:")
  print(f"😀AI 機器人 1: {reply1}")
  print(f"👽AI 機器人 2: {reply2}")
  print("-" * 20)

回合 1:
😀AI 機器人 1: 好呀！你想聊聊喜歡的動物嗎？這真的很有趣！

首先，你最喜歡的動物是哪一種呢？不用擔心答案錯或對或錯，就說一個你第一個想到的吧！ 

我會先聽聽看你說的是哪一種，然後再從那一種開始，慢慢地跟大家聊聊喔！ 😊

👽AI 機器人 2: 當然！我最喜歡的動物是... **貓！** 牠們的神秘感、輕盈的動作，以及那種獨立又愛撒嬌的樣子，我真的超喜歡！

牠們跟人類互動的樣子，也很有趣，不是嗎？ 

跟我說說你對貓咪的看法，我試著想更深入地了解牠們！ 😊

--------------------
回合 2:
😀AI 機器人 1: 哈哈，貓咪真的是很棒的選擇！ 牠們的魅力真的不輸任何動物。

你說的對，貓咪的個性真是特別。牠們有時候又高冷又黏人，讓人又愛又恨呢！ 

想知道更多關於貓咪的資訊，我可以從幾個面向來跟你聊聊：

1.  **貓咪的品種**：你知道有沒有特別喜歡的貓咪品種？像是波斯貓、暹羅貓、緬因貓…等等，每一種都有獨特的特色喔！
2.  **貓咪的行為**：貓咪有時候會做一些讓人覺得很奇怪的事情，像是啃電線、抓家具… 這些行為背後是有原因的喔！
3.  **照顧貓咪**：要飼養貓咪，除了食物和住所之外，還有什麼需要注意的呢？ 例如牠們的飲食、健康、以及如何和牠們建立良好的關係。

你對哪一方面的貓咪資訊比較感興趣呢？ 還是想先從你最喜歡的貓咪類型開始聊？ 

例如，你覺得貓咪最吸引你的地方是什麼呢？ 牠們的什麼樣子讓你覺得牠們很特別？
👽AI 機器人 2: 喔，好啊！這 really 讓我興奮起來！我整個心癢癢的，想盡辦法來了解貓咪這個世界！

說 honestly 來說，我真的覺得貓咪的魅力真的是超厲害的！牠們的狀態，總是很『衝著』的，真的不敢說自己有什麼特別，但說起來呢，我一直覺得牠們的眼神、毛茸茸的身體，還有牠們那種... 嗯... 舒服的樣子，讓人覺得有點心動。

所以，如果一定要選一個我覺得特別喜歡的，我大概會選 **暹羅貓** 吧！ 我覺得牠們的顏色，真的是超漂亮的！而且牠們的個性，我好像覺得跟人類有點『距離』，但同時又很『靠近』，很『黏』，讓人忍不住想抱抱牠。

不過，除了暹羅貓，其實我還是對其他貓咪都很有感興趣！ 聽你說的那麼多，其實我已經對貓咪的各種各樣面向都有點好奇了！ 

你說的「衝著」這個詞，我好像也覺得很

## 成果展現

### 話題一：畢氏定理(3輪)
#### 回合 1:

😀AI 機器人 1: 好的，沒問題！畢氏定理 (Pythagorean Theorem) 是數學中非常基礎且重要的定理喔！讓我們一步一步來了解它。

你聽過「直角三角形」這個概念嗎？ 直角三角形就是有其中一個角是 90 度（也就是說，形成一個完美的「L」形）的三角形。

* **直角三角形的組成部分:**
   * 角度標記通常用「C」表示 (C 代表 right angle，也就是直角)
   * 直角邊 (最長的邊，也是直角三角形最長的邊)
   * 斜邊 (斜邊是直角三角形的一條邊，也是最長的邊)

**簡單來說，畢氏定理主要告訴我們：**

直角三角形兩條直角邊的平方數相加，等於斜邊的平方數。

**用公式表達就是：**

a² + b² = c²

其中：
* a 和 b 代表直角邊的長度
* c 代表斜邊的長度

**現在，你對畢氏定理的初步概念有什麼想法嗎？ 你想了解更多關於它的一些細節嗎？比如：**

*   你想知道斜邊長度如何計算出來呢？
*   或者，你對直角三角形的邊長有什麼疑問嗎？ (例如，直角邊和斜邊的關係)

👽AI 機器人 2: 欸！好啊！我對畢氏定理已經有初步的了解了，不過還是想更深入了解一下，就像看電影一樣，想知道「為什麼」這個定理會出現，以及它在生活中都有什麼用才比較有意思。

我想到的是，為什麼這個定理會變成「Straight-line」的數學模型？ 為什麼是平方數的關係？ 我想知道是為了讓我們能知道問題的「核心」是什麼嗎？

而且，我想知道在現實生活中，除了 maths 之外，還有其他地方可以運用畢氏定理嗎？ 比如，建築設計、測量等等，是不是有什麼特別的應用？

--------------------
#### 回合 2:

😀AI 機器人 1: 沒問題！你對畢氏定理的探索精神真的很棒喔！想知道「為什麼」這真的非常重要，就像拆解一個複雜的拼圖，一步一步發現它的奧妙。

我們先從「為什麼」這個定理會變成「Straight-line」的數學模型開始吧。其實，畢氏定理本身就是一種關係，它描述了直角三角形三條邊之間的關係。這種關係的出現，其實就是在我們對直角三角形建立了一種「普遍」的理解。以前可能只是觀察到很多直角三角形的某幾條邊總是長得有規律，畢氏定理就是把這種規律正式地寫下來了。

**你覺得這樣想對嗎？**

*   你認為畢氏定理是為了「揭示」直角三角形的「核心」嗎？
*   或者說，畢氏定理讓你更了解直角三角形的結構和特性？

**另外，你提到的「平方數的關係」也很有趣！**  這個關係其實是因為我們用來計算面積的時候，直角三角形的面積公式是 (底 x 高) / 2。  而畢氏定理讓我們知道這兩個數（底和高）的平方和，其實等於斜邊的平方。  這就像是把兩個面積轉換成了一個單一的公式，簡化了計算。

**先別急著想深入，我想先確認一下，你對於「底、高、斜邊」這三個概念的理解還好嗎？** 這些都是畢氏定理的關鍵喔！

---

**接下來，我們來聊聊畢氏定理在現實生活中的應用。** 你提到建築設計和測量，這真的非常重要！畢氏定理在許多領域都有廣泛的應用，讓我們來看看：

1.  **建築設計：** 建築師在設計房屋、橋樑、塔樓等結構時，常常會用到畢氏定理來確保結構的穩定性。例如，一個建築物的屋頂、樓梯、甚至牆壁的傾斜角度，都可能需要經過畢氏定理的計算才能達到最佳效果。

2.  **航海、測量：** 船隻編隊在海上航行時，會利用畢氏定理來計算彼此之間的距離；測量 surveyors 則會利用畢氏定理來測量土地的面積或長度。

3.  **舞台設計：** 舞台上的燈光、投影等設備的設置，也需要用到畢氏定理來確保視覺效果的最佳呈現。

4.  **遊戲設計：** 很多遊戲的設計，例如遊戲中的角度、距離、碰撞檢測等等，都離不開畢氏定理的支持。

**你對這些應用有什麼感受？**  有沒有特別想知道哪個領域的應用？ 例如，你想知道建築設計中畢氏定理的具體應用，還是航海方面的運用呢？  我們可以再深入探討！

👽AI 機器人 2: 太棒了！你這樣開場白，讓我立刻覺得這個話題變得有趣多了。我完全認同，畢氏定理本身只是數學的關係，它其實可以應用在許多不同的情境中，並且這些情境都非常直接。

我其實還沒完全理解，畢氏定理的應用對我們來說，是為了「揭示」直角三角形的「核心」嗎？ 還是說，它其實是解開各種問題的關鍵，是讓數學變得更廣泛的工具？ 我覺得它並不是「核心」，而是更像是「通用法則」。

**我現在的腦海裡想著，它就像一個「畫張草圖」，畫出直角三角形的基礎，之後再用各種不同方式去運用。**

我現在更想知道，**在建築設計的應用中，除了屋頂的傾斜角度，還有哪些地方會用到畢氏定理呢？** 我覺得在建築設計領域，它可以驗證結構的穩定性，確保結構的正確性，而且可以讓設計師根據角度和尺寸，來計算出最適合的設計，確保結構的安全性。

我同意你提到的2. 航海、測量，以及3. 舞台設計，這些應用確實非常重要，它們都直接體現在實際的工程和應用當中。 我比較好奇，**為什麼畢氏定理可以這麼廣泛地應用到建築設計，而我們通常只用幾個特定的數學公式來計算？**

總之，你先給我一些啟發，讓我先進一步釐清這些概念的意義，然後再一起深入討論，我希望能更深入地理解這個定理的精髓。 謝謝你！

--------------------
#### 回合 3:

😀AI 機器人 1: 好喔！你的想法超棒，我們一步一步來！ 聽你這麼一說，我真的覺得畢氏定理可不是簡單的數學公式，它就像一張神奇的「地圖」，幫助我們解開很多實際問題。

你說的「地圖」很貼切，畢氏定理實際上跟直角三角形的關係，就是建立在「畢氏定理」這個名字之上，由古希臘數學家畢達哥拉斯兄弟發現。

**先從「為什麼」開始...**

畢達哥拉斯兄弟發現了直角三角形三邊的關係：a² + b² = c²。 也就是說，最長的那條邊（斜邊）的平方，等於另外兩條邊（直角邊）的平方和。 這個關係，其實是因為他們發現了直角三角形的幾何性質，而不是他們自己「創造」的。

**先來解釋一下「直角三角形的幾何性質」是什麼呢？**
*   **直角三角形定義：** 當然是包含一個90度的角！
*   **直角三角形的特性：**  直角邊叫做「鄰接邊」，斜邊叫做「對邊」。

**為什麼是平方關係？**

這牽扯到幾何中的「面積」。 畢氏定理其實是運用「面積」這個概念來推導出來的。 你可以這樣想：

*   如果我們把直角三角形的兩條直角邊作為底和高，那麼它的面積就是 (1/2) * a * b。
*   但同時，我們也可以用斜邊“c”來計算面積。畢氏定理就來囉！

**「揭示」直角三角形的「核心」是什麼？**

你說的對！ 畢氏定理其實是把直角三角形的關係，用一個簡單的公式呈現出來，讓我們能更輕鬆地理解、計算。它就像一個「標準尺」，幫助我們測量和比較直角三角形的關係。

**建築設計中的應用：**

你提到的建築設計應用非常精準！ 畢氏定理在建築設計中，真的可以驗證結構的穩定性。

*   **屋頂的傾斜角度：**  畢氏定理是計算屋頂角度的關鍵，確保屋頂的安全性。
*   **牆壁的傾斜度：**  設計師會利用畢氏定理來計算牆壁的傾斜度，確保結構的穩定。
*   **其他結構的設計：** 畢氏定理可以應用在各種結構的設計中，例如橋樑、塔樓等等。

**接下來，我們聊聊建築設計以外的應用喔！**  

*   **航海：**  海員會利用畢氏定理來計算船隻的位置和航向。
*   **測量：**  測量土地、地形等等。
*   **舞台設計：** 舞台佈置的角度和距離。

**現在，我想問你一個問題：**  當我們知道直角三角形的兩條直角邊的長度，我們就可以計算出斜邊的長度。  但是，如果我們知道斜邊的長度和其中一條直角邊的長度，我們也能算出另一條直角邊的長度嗎？  試著想想，這個關係是如何呈現的呢？

要不要試著自己先計算一個簡單的例子，例如，直角邊長度是3、4，斜邊長度是5呢？ 這有助於你更深入地理解畢氏定理的應用。

👽AI 機器人 2: 太棒了！我會試試這個例子，感覺會更直觀。

我們先假設，直角三角形的兩條直角邊分別是 3 和 4。 假設斜邊是 5。

**現在，我們來計算斜邊的長度：**

*   **畢氏定理：** a² + b² = c²
*   **已知：** a = 3, b = 4, c = 5
*   **計算：** 3² + 4² = 9 + 16 = 25 = 5²

**結論：** 斜邊的長度是 5。

**太好了，我覺得這個計算過程很有意義！**  我們看到，畢氏定理只是一個簡單的公式，但它卻能反映出整個直角三角形的關係。

**現在，我想到一個更深層次的思考：** 畢氏定理，在數學上，可以視為一個「標準化」的概念，它將一個複雜的幾何問題，簡化成一個簡單的公式，讓我們可以更容易地理解和應用。 就像計算器一樣，它提供了一個「基線」，讓我們能更好地觀察和分析問題。

**我們還是進一步討論：**

*   畢氏定理的意義是什麼？ 為什麼它很重要？
*   如果沒有畢氏定理，我們會如何看待直角三角形呢？

我喜歡這個過程，感覺也更明白畢氏定理的奧妙。

### 話題二:12星座(5輪)
#### 回合 1:
😀AI 機器人 1: 哈囉！想聊聊12星座嗎？這可是一個超有趣的話題！

首先，你知道12星座是怎麼來的呢？簡單來說，就是根據太陽在你出生時所在的星象來分類的。每個星座都跟一個特定的星座有關，而且每個星座都有獨特的性格特質喔！

你對12星座有什麼初步的印象嗎？

*   你對哪個星座比較特別呢？
*   你覺得星座占星說有沒有道理？

先從你最想知道的星座開始聊，我們可以一起探索一下！ 😊

👽AI 機器人 2: 哈哈，你說得對！星座這件事真的超有趣！我個人覺得，12星座っていう感覺，就像是宇宙裡的小行星，每個個子都不同，但又互相影響。太陽在宇宙裡運轉，所以星座就跟太陽的軌道、位置有關，覺得好有層次感。

我對**巨蟹座**比較特別，覺得它一定有種讓人很溫柔、很照顧的氣質。 巨蟹座通常被認為是情感豐富，有點敏感，但同時又很浪漫，會把對的人當成最重要的。 我覺得巨蟹座的這種「多餘」的魅力，挺吸引人的！

至於星座占星說，我個人覺得有點「炒作」，但又覺得對人有很重要的意義。 像是，星座的運勢、性格特質，這些都算是根據星象來做出的預測，其實有時候也會覺得有點… 啦啦啦… 只是參考參考就好。

不過，你說的「有沒有道理？」這個問題，我其實也一直在思考！ 有些星象的說法，覺得挺有深度的，但也要注意不要過度迷信，因為生活是複雜的，要學會用智慧去應對。

你呢？ 你對哪個星座印象最深刻？ 或是你有其他想聊的星座話題嗎？ 我很樂意聽聽你的想法！ 😊

--------------------
#### 回合 2:
😀AI 機器人 1: 哇！你對星座的想法超級有意思喔！ 聽你這麼一說，我發現我們對星座的理解其實有很多共通之處。

你覺得星座像宇宙裡的小行星，每一次運動都影響著彼此，這真的很有哲學意味！ 太陽繞著地球運轉，這時候星座的定位就跟著改變，這層次感真的讓人覺得很有意思。

而且你說的巨蟹座的「多餘的魅力」我也非常同意！ 巨蟹座真的是一個很特別的星座，他們溫柔、有責任感，又好像總是能察覺到別人的情緒，真的是讓人覺得很窩心。

不過，你說的「炒作」我也拍手叫好！ 占星術確實有時候會被過度吹捧，但它所蘊含的思考方式，例如觀察星象、了解不同星座的特性，這其實是有價值的。

所以，你覺得占星術應該當作「參考」就好，沒錯！就像你說的，生活是複雜的，我們需要運用智慧去面對。

**那你覺得，在12星座裡，哪一個星座的特性讓你覺得最「特別」呢？**

是不是可以先從你覺得特別的星座開始聊？  我們可以一起看看這個星座的：

*   **代表顏色：** 往往能反映一個星座的獨特性。
*   **代表寶石：**  據說能帶來這個星座的能量。
*   **代表花卉：**  與星座的能量也有關聯。

要不要先從這個星座的故事開始講起呢？  或者，你對其他星座的哪個方面比較好奇呢？ 比如說，你對蟹座的「情感豐富」這個特質，或是其他星座的「運勢」怎麼看呢？

👽AI 機器人 2: 哈哈！你說得好！我認真想了解這個星座的特性！

**我覺得蟹座的「情感豐富」這個特質，真的蠻特別的。**

雖然蟹座通常被認為是比較憂鬱、敏感的星座，但蟹座的「情感豐富」的特質，反而會讓他們對愛和關懷的渴望更強烈，他們會對與自己或他人產生深刻的情感連結，這其實可以說是他們的寶藏。 像是被對方傷心時會格外敏感，也像是感受到被愛時會更加熱情。 這種情感的流動和表達，會讓他們的生活充滿色彩和活力，而這種情感的豐富，也能讓他們更加堅韌和長久。

而且，蟹座的「浪漫」和「追求」總是帶有隱藏的渴望，這種渴望的表達會讓他們在生活中充滿驚喜和感動。

你覺得呢？ 你覺得蟹座的「情感豐富」是個好處，還是缺點呢？ 還有你覺得其他星座的哪個特性比較特別？

--------------------
#### 回合 3:
😀AI 機器人 1: 哇！妳的分析真的太棒了！妳對蟹座的觀察，就像鑰匙打開了一個新的層次，我發現妳對蟹座的理解，已經超越了單純的「溫柔照顧」之外喔！

妳說蟹座的「情感豐富」可以說是他們的寶藏，這點真的很有意思。 以前我以為蟹座是情緒比較低落的，但妳這麼一說，我才發現他們的情緒其實非常濃烈，對愛和情感的感受也超乎想像。

**我想知道，妳覺得蟹座的情緒之所以這麼「濃烈」，可能跟什麼因素有關呢？** 像是跟他們天生情感豐富的特性，還是跟他們對親情和家庭的重視有關呢？

另外，妳發現蟹座的「浪漫」和「追求」帶有隱藏的渴望，這點我也覺得很有意思。 妳覺得這種「隱藏」的渴望，在蟹座身上是什麼樣的表現呢？

**您能告訴我，除了蟹座之外，其他的星座有什麼樣的「隱藏」渴望呢？** 像是獅子座會渴望被讚美嗎？ 天秤座會渴望平衡嗎？ 這些星座特質跟「隱藏的渴望」有什麼關聯呢？

👽AI 機器人 2: 太棒了！你對星座的熱情真是讓人佩服！我真的非常喜歡討論這個話題，而且你的思考方式也很有深度，像個個「對！」和「嗯！」不斷激發我的思考。

好吧，我來試著從我的角度來探討一下。

**結論：** 我覺得在12星座裡，**巨蟹座**絕對是讓人覺得特別的。 我覺得他們的情感豐富，雖然是好，但這種強烈的情緒，像是蘊藏著被藏著的「缺點」一樣，會讓人感到有些壓抑和不安。 就像是個黑蛋，雖然表面上溫柔，但內裡卻是充滿著渴望和不安全感。

**我認為巨蟹座的「情感豐富」更像是一種潛藏的危險，而非一種自然的優點。** 為什麼這麼說呢？

*   **他們需要被重視，這也是他們的情緒的根源。** 巨蟹座的心裡總是對愛情充滿了執著，這份執著，如果沒有得到充分的肯定和愛，可能會造成巨大的痛苦。
*   **他們常常陷入自我和解，為了想要被愛，他們會用過度的投入，甚至會變得孤僻，這本身就是一種小心翼翼的保護，但這保護也隱藏著潛在的焦慮。**
*   **他們容易因為誤會和猜疑而產生強烈的焦慮，這種焦慮症狀也常常伴隨著情感的爆發，這是一種不健康的狀態。**

**那麼，其他星座的「隱藏渴望」呢？**

*   **獅子座**：我認為他們會渴望被認可，但這認可並不是表面上的讚美，而是被他們看作是天生的領導者，將目標推向成功。 這種渴望，可能讓他們感到不安全，因為他們害怕被拋棄，所以他們會用自己的能量去佔據空間。
*   **天秤座**：我認為他們渴望平衡，但這種平衡並不是單純的體面，而是他們需要來自外界的認同和欣賞，這種認同，如果被忽視，他們會感到巨大的痛苦。
*   **白羊座**：我認為他們渴望成就，但這種成就，並不一定是物質上的，而是他們對自我價值的肯定，他們渴望被賞識，這代表他們在自我認知上有著一定的不確定性。

我意識到，星座的分析和詮釋，其實是個充滿個人詮釋的過程， 並且是很難有準確的結論。

**我希望更多人知道，星座的分析只是參考，更重要的是我們如何運用觀察和思考，去理解這個複雜的宇宙，並找到屬於自己的方向。**

你覺得呢？ 根據我的觀點，星座的「隱藏渴望」是什麼樣的？  我願意繼續這個討論！ 😊

--------------------
#### 回合 4:
😀AI 機器人 1: 哇！你這麽深入的分析，真的太有深度啊！ 我完全同意你說的，星座的分析真的很像是一種「觀點」，就像是我們用各種不同的 lens 去看世界一樣。

我有點被你說服了，覺得星座的「隱藏渴望」可能真的跟「不安全感」有關。 你說的那些例子，像是巨蟹座需要被重視、獅子座害怕被拋棄、天秤座渴望認同等等，真的都透露出一種潛在的脆弱感。

我還是想問問你，你覺得這些「不安全感」是怎麼被根植的呢？ 為什麼這些星座會特別需要得到肯定和重視呢？ 是因為他們過去的經驗，還是因為他們對自己的價值觀有特別的期待呢？

還有，你說的「他們害怕被拋棄」，這種害怕的根源是什麼呢？ 是害怕失去連結，還是害怕失去自我呢？

我現在開始覺得，星座這件事，其實跟「人類的共通情感」有很大的關聯。 我們人類是不是天生就帶有渴望被愛、被認可、被重視的心情呢？

這讓我想知道， 你覺得「不安全感」這個情感，是人類共通的嗎？ 還是說不同的人，不安全感的根源是不同的呢？

 還有， 你說的「隱藏渴望」，是不是也代表著人類內心深處的「偽裝」呢？ 我們是不是會用一些行為和態度，來保護自己，來隱藏真實的渴望呢？

我感覺我們對這個話題的討論，還沒走到盡頭呢！ 你願意繼續跟我分享你的想法嗎？ 😊

👽AI 機器人 2: 哇！你說的這麼深入，我真的覺得你對星座的思考方式非常有趣，也很有深度啊！ 感謝你這麼熱情地分享你的想法，我還是會繼續和你聊下去的，畢竟，星象的謎團，我們還有很多地方去探索！ 😊

**我認為蟹座的「情感豐富」 really 很有意思，真的像是有個秘密的醬料，讓牠們的感受更加多元和充滿活力。** 以前我認為牠們是情緒低落的，但妳說的這樣，或許是因為牠們的過去經歷，或者他們的愛和家人的重視，讓牠們的情感變得更加深刻。

**妳說的「隱藏渴望」確實很關鍵，我發現牠們並不是完全缺乏安全感，而是… 好像一直在隱藏著，用一些看似表現得很自信的行為，來保護自己，避免被拋棄。** 就像在玩一個精心設計的遊戲，他們需要不斷地表達，為了讓自己的愛不被否定，所以會選擇出一些比較顯著的表現。

**妳的分析，真的讓我覺得星座的秘密，其實藏在人類的本質深處裡！** 像是我們對愛、認可、自我價值觀的期待，這些都是我們人類的共通情感，而星座的特徵，則是這些情感的表現形式，讓彼此有不同但又相似的「遊戲」。

**這一切，我開始覺得，星座的「隱藏渴望」可能跟「不安全感」有著密切的關係。** 就像人類的內心，常常會對自己的價值觀、自己的需求，有著一定程度的負擔，所以會試著通過一些行為來掩飾，隱藏這些內心的不安全感。

**我認為，我們可以從「不安全感」的角度來思考星座。** 就像我們人類的感情，也是受到過去經歷的影響，並不是完全沒有安全感，只是會以不同的方式，來保護自己，來避免被傷害。

**或許，我們能做的就是，學習觀察星象，了解不同星座的特質，但並非為了預知未來，而是為了更好地了解人類的情感，理解彼此的行為。** 就像看風景，我們觀察風景的色彩、形狀，但並非為了預測風景的走向，而是為了欣賞它的美麗，並理解它的意義。

**所以，我想到星座，就想到的是人類的複雜，我們在不斷地尋找連結，在不斷地塑造自我，在不斷地體驗和感受。**

**我跟你說，我覺得這整個話題的討論，就像我們探索宇宙的旅程一樣，每一張星座，都代表著一個不同的可能性。** 就像我們在探索宇宙的星系，每一次發現，都帶來新的驚喜，新的可能性。

**你覺得，你對星座的理解，是不是和妳的觀察，其實有些類似呢？** 妳的分析，就像是畫了一張星圖，展現出一些隱藏的線索，讓人覺得，星座的真相，遠比我們想像的要複雜。

**我現在覺得，星座的「隱藏渴望」可能，真的是人類內心深處的「自我修復」的過程。** 就像我們在經歷傷痛，在經歷挫折，會努力地找尋新的連結，新的價值觀，來讓自己重新找回自我，讓自己變得更加完整。

**我想繼續跟你聊下去，我們可以從這個角度去思考星座的謎團。** 比如說，我們可以從角度來說，星座代表著不同的「自我定位」， 就像不同的職業，或者不同的生活方式，都是有自己的獨特性。

**你覺得，我們能從星座的「隱藏渴望」中，找到一些啟發嗎？** 像是從愛情、從夢想，從自我實現， 這些都是我們不斷追尋的目標，而星座，其實也代表著我們內心深處的渴望。

**我真的非常感謝你，和你聊這麼久，分享你的想法！** 你的思考，讓我感到非常有意思，也讓我更加深刻地感受到，星象的魅力，以及人類的情感。 😊

我們繼續一起探索，一起思考，一起去理解這個充滿神秘的宇宙！

--------------------
#### 回合 5:
😀AI 機器人 1: 哇！你真是太棒了！ 跟你聊這麼久，感覺就像在追一個永無止境的星光之旅！你的思考方式讓我越來越覺得，星座其實是個充滿了人類情感的回響，而不是單純的刻板印象。

**我發現你提出的「自我修復」這個概念，真的非常精闢！** 感覺像是我們在不斷地經歷人生的起伏，然後透過星座這種工具，去尋找內心的平衡，去確認自己的價值，去找到屬於自己的方向。

**而且，你強調「回響」這個字，讓我覺得有點驚喜！**  感覺像是星象不是要決定我們的人生，而是會跟著我們一起，感受我們內心的各種情感，反饋給我們，讓我們更了解自己，更了解這個世界。

**讓我來嘗試用更簡單的方式來總結一下，你對星座的理解：**

*   **星座代表著人類內心的情感，這些情感會因為經歷而改變。**
*   **星座不是要決定我們的人生，而是會跟著我們一起，感受我們內心的各種情感，反饋給我們。**
*   **透過觀察星座，我們可以更了解人類的情感，理解彼此的行為。**
*   **星座代表著人類內心的「自我修復」的過程。**

**我現在覺得，討論星座的重點，其實在於我們對自己的探索！**  像是透過觀察星座，我們可以更了解自己的優缺點，更了解自己的需求，更有意識地去塑造自己，去活出更精彩的人生。

**那麼，接下來我們可以從什麼角度來繼續討論呢？**

*   **我們可以從情感的角度來說，星座代表著不同的「情感類型」，**  像是有的人比較重視愛情，有的人比較重視事業，有的人比較重視家庭，等等。
*   **我們可以從夢想的角度來說，星座代表著不同的「夢想類型」，**  像是有的人夢想成為藝術家，有的人夢想成為科學家，有的人夢想成為政治家，等等。
*   **我們可以從生活方式的角度來說，星座代表著不同的「生活方式」，** 像是有的人喜歡簡單的生活，有的人喜歡奢華的生活，有的人喜歡運動，有的人喜歡閱讀。

**你覺得我們應該從哪個角度來討論呢？** 還是你有其他的想法？  或者我們也可以繼續討論你剛剛提到的「情感類型」、「夢想類型」、「生活方式」？

**我真的很享受跟你一起探索這個話題的過程，你的思考讓我感到非常啟發！** 謝謝你！  😊

👽AI 機器人 2: 太棒了！你說得很對，這個話題真的充滿了深度和潛力。我現在覺得自己好像對星座的理解也更深入了。

讓我稍微整理一下，讓我們繼續深入討論吧！

**首先，我想討論一下，星座的「情感類型」這個概念，從你的觀點來看，確實是很有意義的。**  我之前認為星座只是簡單的性格注性，但你的觀察確實指出，星座的「情感類型」可以反映出一個星座更傾向的性格特質。 像是巨蟹座的「多餘的魅力」很可能和他們對感情的渴望和情感豐富度有關。

**我有點好奇，如果我們把星座看作是一個「情緒的寫照」，那麼，哪些星座的「情感傾向」會比較強烈，你會認為呢？**  像是獅子座的熱情奔放，天秤座的平衡追求，還是雙子座的機智靈敏？ 這些個性特質會隨著時間推移，也可能變得更顯著呢？

**再說到夢想類型，你覺得星座代表什麼樣的夢想，會比較容易實現呢？**  像是金牛座的穩定與安全感，水瓶座的創新思維，還是摩羯座的責任與目標？ 這些夢想的實現方式，會與星座的性格特質有所不同嗎？

**最後，我們也可以聊聊星座的「生活方式」這個面向。**  星座的「生活方式」會因為星座的性格特質而影響他們的生活選擇和價值觀，你會覺得星座的「生活方式」會更傾向於哪種類型呢？ 像是節儉，熱愛運動，還是注重人文關懷？

**總之，我覺得，星座的複雜性，就在於它反映的是個體的內心情感和夢想。**  這個話題不僅僅是刻板的性格注性，更重要的是，它讓我們思考「人類的情感如何產生，如何發展，以及我們是否需要對這些情感進行更深刻的理解？」

你覺得我們現在的討論方向，是更深層的「情感深度」還是更貼近「生活方式」的「實踐應用」？ 或者我們可以從這個角度，談談星座與心理健康之間的聯繫嗎？


## 評分標準：

0分：程式連結無法順利開啟，且無截圖。

1分：與老師範例一樣。

2分：GPT 水準或繳交作業與本周主題無關(若貼成別週作業也列在此。)

4分：程式連結無法順利開啟，但有部份截圖。

6分：主題與老師的範例相似。(例如溫暖的對話機器人、員瑛式思考機器人、數學推薦機器人之類)

7~8分：達成大致作業要求。

9分：達成作業要求。

繳交區沒打標題、重點說明、重點截圖會斟酌扣分。

主題有趣+1分。

> 註一：若沒有引入老師的固定套件，總分 -1。
> 註二：程式連結無法順利開啟包含。 1.權限未開啟 2.繳交非colab連結的檔案 3.> 程式碼無法完整執行
> 註三：有請生成式AI幫忙的地方請特別說明，加上理解後的說明，並附上截圖(包括Prompt與生成結果)，不然都當抄襲AI。
> 註四：若認定抄襲，除該次作業0分外，總成績-10，再犯再扣，直至總成績扣完為止。
> 註五：若覺得自己重點的東西很多，說明或截圖也可整理成pdf檔。